# Importing and Setting up the API

In [1]:
# Note: this was necessary even though I installed spotipy via my cmd because the shell and this kernel use different versions
# of python. So in order to install spotipy here, we need to install it using sys.executable and not regular python3.
import sys
!{sys.executable} -m pip install spotipy


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\neela\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import cred
import time

For our web app, we're going to use the **Client Credentials** flow. This project is relatively small in scale, so this flow allows us to access a user's data without the need to implement refresh tokens.

In [13]:
# Authenticating (without user credentials)
cid = cred.client_ID
secret = cred.client_SECRET

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=10)

In [14]:
# Random Test
weeknduri = 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ' # you don't need the 'spotify:artist:'
results = sp.artist_top_tracks(weeknduri) # dictionary
for i, track in enumerate(results['tracks'][:10]):
    print(f'{i+1}. ' + track['name'])

1. Die For You (with Ariana Grande) - Remix
2. Creepin' (with The Weeknd & 21 Savage)
3. Die For You
4. Starboy
5. Blinding Lights
6. Save Your Tears
7. I Was Never There
8. Reminder
9. Call Out My Name
10. Save Your Tears (Remix) (with Ariana Grande) - Bonus Track


In [15]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split('/')[-1].split('?')[0] # extracts the URI from the link
playlist_URI

track_uris = []
for track in sp.playlist_tracks(playlist_URI)['items']:
    track_uris.append(track['track']['uri'])

# track_uris

In [16]:
sp.audio_features(track_uris[0]) # this is method we can use to get features of songs

[{'danceability': 0.707,
  'energy': 0.681,
  'key': 0,
  'loudness': -4.325,
  'mode': 1,
  'speechiness': 0.0668,
  'acousticness': 0.0632,
  'instrumentalness': 5.15e-06,
  'liveness': 0.0322,
  'valence': 0.646,
  'tempo': 117.999,
  'type': 'audio_features',
  'id': '4DHcnVTT87F0zZhRPYmZ3B',
  'uri': 'spotify:track:4DHcnVTT87F0zZhRPYmZ3B',
  'track_href': 'https://api.spotify.com/v1/tracks/4DHcnVTT87F0zZhRPYmZ3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4DHcnVTT87F0zZhRPYmZ3B',
  'duration_ms': 200455,
  'time_signature': 4}]

# Getting the data

In [18]:
import pandas as pd
import json

In [19]:
data_path = 'mpd.slice.0-999.json'
json_dict = json.loads(open(data_path).read()) # json.loads() deserializes a string representing json object and returns a python dictionary
# note: json.load(open(data_path)) would've done the same thing, loads works on strings and load works on file objects

playlists = json_dict['playlists'] # list of 
df = pd.json_normalize(playlists, record_path='tracks', meta='name') # record_path looks at 'tracks' in every playlist, meta grabs the playlist name

# df.to_csv('raw_data.csv')
df

# pd.read_json('mpd.slice.0-999.json').head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks
...,...,...,...,...,...,...,...,...,...
67498,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge,thinking of you
67499,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you
67500,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you
67501,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you


In [20]:
quick = pd.DataFrame(columns=["lengths"])
for i in range(len(playlists)):
    quick.loc[len(quick.index)] = len(playlists[i]['tracks'])

quick.iloc[:,0].mean(axis=0)

67.503

In [21]:
# Test
fd = pd.json_normalize(playlists, record_path='tracks')
fd.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


# Extracting the features from the data

In [22]:
data_path = 'raw_data.csv'
df2 = pd.read_csv(data_path)
# df2['track_uri'] = df2['track_uri'].apply(lambda x: re.findall(r'\w+$', x)[0]) Not necessary lmao
# df2 = df2[:100]
df2
# for numrow in range(len(df)):
#     df['track_uri'][i]
# print(df2.shape)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks
2,2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks
3,3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks
4,4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks
...,...,...,...,...,...,...,...,...,...,...
67498,67498,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge,thinking of you
67499,67499,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you
67500,67500,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you
67501,67501,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you


We'll add in the audio features as columns, as well as the artist's popularity, the genres of the song, and the track's popularity.

In [23]:
import re
def features(trackuri):
    featuredict = sp.audio_features(trackuri)[0]
    artistid = sp.track(trackuri)['artists'][0]['id']
    artistpop = sp.artist(artistid)['popularity'] # add in artist popularity
    featuredict['artistpop'] = artistpop
    trackpop = sp.track(trackuri)['popularity'] # add in track popularity
    featuredict['trackpop'] = trackpop
    genres = sp.artist(artistid)['genres']
#     genres_und = []
#     for genre in genres:
#         if ' ' in genre:
#             genres_und.append(genre.replace(' ', '_'))
#         else: genres_und.append(genre)
    if genres:
        featuredict['genres'] = ' '.join(re.sub(' ','_',genre) for genre in genres) # short version
    else: 
        featuredict['genres'] = 'unknown'

    return featuredict

# print(type(sp.artist('2wIVse2owClT7go1WT98tk')['popularity']))
# print(sp.artist('2wIVse2owClT7go1WT98tk')['genres'])
# print(sp.track('0UaMYEvWZi0ZqiDOoHU3YI')['artists'][0]['id'])
print(features('spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'))

{'danceability': 0.904, 'energy': 0.813, 'key': 4, 'loudness': -7.105, 'mode': 0, 'speechiness': 0.121, 'acousticness': 0.0311, 'instrumentalness': 0.00697, 'liveness': 0.0471, 'valence': 0.81, 'tempo': 125.461, 'type': 'audio_features', 'id': '0UaMYEvWZi0ZqiDOoHU3YI', 'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI', 'duration_ms': 226864, 'time_signature': 4, 'artistpop': 74, 'trackpop': 68, 'genres': 'dance_pop hip_hop hip_pop pop_rap r&b rap urban_contemporary virginia_hip_hop'}


In [24]:
featurelist = [] # list of dictionaries

for i, track in enumerate(df2[:1000]['track_uri']):
    if i%5==0: print(i, end=' ')
    try: featurelist.append(features(track))
    except: continue
    if i%100==0: 
        print("sleeping 30 sec...")
        time.sleep(10)
        print("20 sec....")
        time.sleep(10)
        print("10 sec...")
        time.sleep(10)
        
    
featuredf = pd.DataFrame(featurelist)
featuredf.rename({"uri":"track_uri"}, axis='columns', inplace=True)
featuredf

0 sleeping 30 sec...
20 sec....
10 sec...
5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 sleeping 30 sec...
20 sec....
10 sec...
105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 200 sleeping 30 sec...
20 sec....
10 sec...
205 210 215 220 225 230 235 240 245 250 255 260 265 270 275 280 285 290 295 300 sleeping 30 sec...
20 sec....
10 sec...
305 310 315 320 325 330 335 340 345 350 355 360 365 370 375 380 385 390 395 400 sleeping 30 sec...
20 sec....
10 sec...
405 410 415 420 425 430 435 440 445 450 455 460 465 470 475 480 485 490 495 500 sleeping 30 sec...
20 sec....
10 sec...
505 510 515 520 525 530 535 540 545 550 555 560 565 570 575 580 585 590 595 600 sleeping 30 sec...
20 sec....
10 sec...
605 610 615 620 625 630 635 640 645 650 655 660 665 670 675 680 685 690 695 700 sleeping 30 sec...
20 sec....
10 sec...
705 710 715 720 725 730 735 740 745 750 755 760 765 770 775 780 785 790 795 800 sleeping 30 sec...
20 sec....
10 sec...
805 810 815 820 82

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature,artistpop,trackpop,genres
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,68,dance_pop hip_hop hip_pop pop_rap r&b rap urba...
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,82,dance_pop pop
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,...,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,87,20,dance_pop pop r&b
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,...,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,81,77,dance_pop pop
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,...,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.799,0.586,11,-6.735,1,0.0377,0.162000,0.000002,0.1930,0.549,...,audio_features,5HuqzFfq2ulY1iBAW5CxLe,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,https://api.spotify.com/v1/tracks/5HuqzFfq2ulY...,https://api.spotify.com/v1/audio-analysis/5Huq...,227833,4,76,49,dance_pop edm pop pop_dance pop_rap tropical_h...
996,0.543,0.668,0,-7.396,1,0.0272,0.000724,0.000080,0.3220,0.428,...,audio_features,5eqz8gTDU13s7x2DZKiQA9,spotify:track:5eqz8gTDU13s7x2DZKiQA9,https://api.spotify.com/v1/tracks/5eqz8gTDU13s...,https://api.spotify.com/v1/audio-analysis/5eqz...,227267,4,50,0,anthem_worship ccm christian_alternative_rock ...
997,0.789,0.555,0,-5.900,1,0.0410,0.026000,0.000135,0.2150,0.713,...,audio_features,3V3iy4K6paycRmTyrjQVwi,spotify:track:3V3iy4K6paycRmTyrjQVwi,https://api.spotify.com/v1/tracks/3V3iy4K6payc...,https://api.spotify.com/v1/audio-analysis/3V3i...,223405,4,51,45,neo_mellow
998,0.705,0.780,7,-6.267,1,0.0805,0.456000,0.001520,0.1150,0.457,...,audio_features,0qt5f5EL92o8Snzopsv0en,spotify:track:0qt5f5EL92o8Snzopsv0en,https://api.spotify.com/v1/tracks/0qt5f5EL92o8...,https://api.spotify.com/v1/audio-analysis/0qt5...,218228,4,63,70,australian_indie australian_pop folk-pop pop p...


In [25]:
# df2small = df2.head(100)
# final_100 = pd.merge(df2small, featuredf, how='outer')
# final_100
featuredf.head(2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature,artistpop,trackpop,genres
0,0.904,0.813,4,-7.105,0,0.121,0.0311,0.00697,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,68,dance_pop hip_hop hip_pop pop_rap r&b rap urba...
1,0.774,0.838,5,-3.914,0,0.114,0.0249,0.02500,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,82,dance_pop pop


In [15]:
df2.head(2)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks


In [26]:
master = pd.merge(df2[:1000], featuredf, on='track_uri')
master.drop(columns=['Unnamed: 0','pos'], inplace=True)
# cols = master.columns.tolist()
# cols = [cols[7]] + cols[0:7] + cols[8:]
# master = master[cols]

In [27]:
# master.to_csv('out2.csv')
print(master.shape)
print(df2.shape)
print(featuredf.shape)

(1064, 28)
(67503, 10)
(1000, 21)


In [34]:
master['id'].isna().sum()

0

In [29]:
master.to_csv('processed1000.csv')